# Semantic Kernel

W tym przykładzie kodu użyjesz [Semantic Kernel](https://aka.ms/ai-agents-beginners/semantic-kernel), frameworka AI, aby stworzyć podstawowego agenta.

Celem tego przykładu jest pokazanie kroków, które później wykorzystamy w dodatkowych przykładach kodu podczas implementacji różnych wzorców agentowych.


## Zaimportuj potrzebne pakiety Pythona


In [ ]:
import os 
from typing import Annotated
from openai import AsyncOpenAI

from dotenv import load_dotenv

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.functions import kernel_function

## Tworzenie klienta

W tym przykładzie użyjemy [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) do uzyskania dostępu do LLM.

`ai_model_id` jest zdefiniowany jako `gpt-4o-mini`. Spróbuj zmienić model na inny dostępny w marketplace GitHub Models, aby zobaczyć różne wyniki.

Aby użyć `Azure Inference SDK`, który jest wykorzystywany dla `base_url` w GitHub Models, skorzystamy z konektora `OpenAIChatCompletion` w Semantic Kernel. Istnieją również inne [dostępne konektory](https://learn.microsoft.com/semantic-kernel/concepts/ai-services/chat-completion), które pozwalają używać Semantic Kernel z innymi dostawcami modeli.


In [ ]:
import random   

# Define a sample plugin for the sample

class DestinationsPlugin:
    """A List of Random Destinations for a vacation."""

    def __init__(self):
        # List of vacation destinations
        self.destinations = [
            "Barcelona, Spain",
            "Paris, France",
            "Berlin, Germany",
            "Tokyo, Japan",
            "Sydney, Australia",
            "New York, USA",
            "Cairo, Egypt",
            "Cape Town, South Africa",
            "Rio de Janeiro, Brazil",
            "Bali, Indonesia"
        ]
        # Track last destination to avoid repeats
        self.last_destination = None

    @kernel_function(description="Provides a random vacation destination.")
    def get_random_destination(self) -> Annotated[str, "Returns a random vacation destination."]:
        # Get available destinations (excluding last one if possible)
        available_destinations = self.destinations.copy()
        if self.last_destination and len(available_destinations) > 1:
            available_destinations.remove(self.last_destination)

        # Select a random destination
        destination = random.choice(available_destinations)

        # Update the last destination
        self.last_destination = destination

        return destination

In [ ]:
load_dotenv()
client = AsyncOpenAI(
    api_key=os.environ.get("GITHUB_TOKEN"), 
    base_url="https://models.inference.ai.azure.com/",
)

# Create an AI Service that will be used by the `ChatCompletionAgent`
chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)

## Tworzenie Agenta

Poniżej tworzymy Agenta o nazwie `TravelAgent`.

W tym przykładzie używamy bardzo prostych instrukcji. Możesz zmienić te instrukcje, aby zobaczyć, jak agent reaguje inaczej.


In [ ]:
agent = ChatCompletionAgent(
    service=chat_completion_service, 
    plugins=[DestinationsPlugin()],
    name="TravelAgent",
    instructions="You are a helpful AI Agent that can help plan vacations for customers at random destinations",
)

## Uruchamianie Agenta

Teraz możemy uruchomić Agenta, definiując wątek typu `ChatHistoryAgentThread`. Wszelkie wymagane wiadomości systemowe są przekazywane do argumentu słowa kluczowego `messages` w funkcji invoke_stream agenta.

Po ich zdefiniowaniu tworzymy `user_inputs`, które będą tym, co użytkownik wysyła do agenta. W tym przypadku ustawiliśmy tę wiadomość na `Plan me a sunny vacation`.

Śmiało zmień tę wiadomość, aby zobaczyć, jak agent odpowiada w inny sposób.


In [ ]:
async def main():
    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: ChatHistoryAgentThread | None = None

    user_inputs = [
        "Plan me a day trip.",
    ]

    for user_input in user_inputs:
        print(f"# User: {user_input}\n")
        first_chunk = True
        async for response in agent.invoke_stream(
            messages=user_input, thread=thread,
        ):
            # 5. Print the response
            if first_chunk:
                print(f"# {response.name}: ", end="", flush=True)
                first_chunk = False
            print(f"{response}", end="", flush=True)
            thread = response.thread
        print()

    # Clean up the thread
    await thread.delete() if thread else None

await main()


---

**Zastrzeżenie**:  
Ten dokument został przetłumaczony za pomocą usługi tłumaczenia AI [Co-op Translator](https://github.com/Azure/co-op-translator). Chociaż dokładamy wszelkich starań, aby tłumaczenie było precyzyjne, prosimy pamiętać, że automatyczne tłumaczenia mogą zawierać błędy lub nieścisłości. Oryginalny dokument w jego rodzimym języku powinien być uznawany za źródło autorytatywne. W przypadku informacji o kluczowym znaczeniu zaleca się skorzystanie z profesjonalnego tłumaczenia przez człowieka. Nie ponosimy odpowiedzialności za jakiekolwiek nieporozumienia lub błędne interpretacje wynikające z użycia tego tłumaczenia.
